In [1]:
#clone HairFastGAN github
!git clone https://github.com/AIRI-Institute/HairFastGAN.git
%cd HairFastGAN

d:\PYTHON\MiAI\HairFastGAN\HairFastGAN


fatal: destination path 'HairFastGAN' already exists and is not an empty directory.
c:\Users\trung\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
#Install ninja - small build system to run C++, C
# Install Ninja - small build system https://github.com/ninja-build/ninja
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

'wget' is not recognized as an internal or external command,
operable program or batch file.


Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app
Sudo is disabled on this machine. To enable it, go to the ]8;;ms-settings:developers\Developer Settings page]8;;\ in the Settings app


In [ ]:
# Install lib
!pip install pyngrok streamlit pillow==10.0.0 face_alignment dill==0.2.7.1 addict fpie git+https://github.com/openai/CLIP.git -q


In [ ]:

!pip install torchvision


In [ ]:
# Download pretrain
!git clone https://huggingface.co/AIRI-Institute/HairFastGAN
!cd HairFastGAN && git lfs pull && cd ..
!mv HairFastGAN/pretrained_models pretrained_models
!mv HairFastGAN/input input
!rm -rf HairFastGAN

In [ ]:
# Khai báo các thư viện
from pathlib import Path #làm việc với đường dẩn
from hair_swap import HairFast, get_parser # 2 thư viện cần thiết của HairFastGAN

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torchvision.transforms as T
import torch


In [ ]:
# Khởi tạo model để test
model_args = get_parser()
hair_fast = HairFast(model_args.parse_args([]))


In [ ]:
#Hàm hiển thị ảnh theo đúng thứ tự
# Display function
def display_images(images=None, **kwargs):
    is_titles = images is None
    images = images or kwargs

    grid = gridspec.GridSpec(1, len(images))
    fig = plt.figure(figsize=(20, 10))

    for i, item in enumerate(images.items() if is_titles else images):
        title, img = item if is_titles else (None, item)

        img = T.functional.to_pil_image(img) if isinstance(img, torch.Tensor) else img
        img = Image.open(img) if isinstance(img, str | Path) else img

        ax = fig.add_subplot(1, len(images), i+1)
        ax.imshow(img)
        if title:
            ax.set_title(title, fontsize=20)
        ax.axis('off')

    plt.show()

In [ ]:

input_dir = Path("HairFastGAN\Images")
face_path = input_dir / 'LISA.jpg'
shape_path = input_dir / 'jisoo.png'
color_path = input_dir / 'ROSE.png'

final_image, face_align, shape_align, color_align = hair_fast.swap(face_path,shape_path,color_path, align=True)

%matplotlib inline
display_images(final_result=final_image, input_face=face_align, hair_shape=shape_align, hair_color=color_align)



# input_dir = Path("/content/HairFastGAN/input")
# face_path = input_dir / 'Lanh.jpg'
# shape_path = input_dir / '4.jpg'
# color_path = input_dir / '3.jpg'

# final_image, face_align, shape_align, color_align = hair_fast.swap(face_path,shape_path,color_path, align=True)

# %matplotlib inline
# display_images(final_result=final_image, input_face=face_align, hair_shape=shape_align, hair_color=color_align)


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from io import StringIO
from pathlib import Path

# Init model
from pathlib import Path
from hair_swap import HairFast,get_parser
# import requests
from io import BytesIO
from PIL import Image
# from functools import cache

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torchvision.transforms as T
import torch


if "model" not in st.session_state.keys():
    model_args = get_parser()
    hair_fast = HairFast(model_args.parse_args([]))
    st.session_state["model"] = hair_fast

hair_fast = st.session_state["model"]

st.set_page_config(
        page_title="Virtual Hair",
)

st.title("Salon ABC - Virtual Hair Style")
input_face = st.file_uploader("Chọn ảnh khuôn mặt bạn")
hair_shape = st.file_uploader("Chọn ảnh mẫu tóc")
hair_color = st.file_uploader("Chọn ảnh màu tóc")


if st.button("Change me!!!", type="primary"):
    # st.image('result.png', caption='Sunrise by the mountains')

    if input_face is not None and hair_shape is not None and hair_color is not None:
        # To read file as bytes:
        save_folder = '/content/HairFastGAN/upload'
        face_path = Path(save_folder, input_face.name)
        with open(face_path, mode='wb') as w:
            w.write(input_face.getvalue())
        shape_path = Path(save_folder, hair_shape.name)
        with open(shape_path, mode='wb') as w:
            w.write(hair_shape.getvalue())
        color_path = Path(save_folder, hair_color.name)
        with open(color_path, mode='wb') as w:
            w.write(hair_color.getvalue())

        final_image, face_align, shape_align, color_align =  hair_fast.swap(face_path, shape_path, color_path, align=True)
        # Save output to file ressult.png
        save_path = Path(save_folder, 'result.png')
        final_image = T.functional.to_pil_image(final_image)
        # final_image.save(save_path)
        st.image(final_image, caption='Render result')
        del final_image, face_align, shape_align, color_align




In [ ]:
# Run app.py by streamlit
!streamlit run app.py&>/dev/null&
!pgrep streamlit

In [ ]:
# Expose by grok
!ngrok authtoken '*******************************************'

In [ ]:
#streamlit expose ra cổng 8501 và ta dùng ngrok để expose nó ra
!ngrok http 8501